# Solución tarea 2 pregunta 2

# Funciones auxiliares
Antes de escribir nuestra clase necesitamos el test de primalidad y el algoritmo extendido de euclides.
Para el test de primalidad primero necesitamos revisar si un número es potencia de otro.

In [1]:
def es_potencia(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

Probemos algunos casos

In [2]:
assert not es_potencia(8 ** 30 + 7)
assert es_potencia(3 ** 15)
assert es_potencia(12 ** 10)
assert not es_potencia(7 ** 10 + 1)
assert es_potencia(5 ** 3)

Parece portarse bien, definamos ahora el algoritmo extendido de euclides.

In [3]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    r, s, t = _extended_euclid_base(b, a)
    return r, t, s

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

Pasamos finalmente a definir la función que nos dice que un número es probablemente un primo

In [4]:
import random

In [5]:
def _is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if es_potencia(n):
        return False
    
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

Probemos algunos casos

In [6]:
assert not _is_probably_prime(12387568637)
assert _is_probably_prime(5943632362508814456797384433006100442712303295066940614569354936549874999082678378231629906729379134167935471382621311620276545251597436711454168850267595109678077983960376792735878876067066338864239372227790339203350191408856924700453890622245349547304896138668552188577288047417779378700982792791819866553113608966810109430765067528429902116607213626746562172730714525439765422832045628189761714003)

Pareciera estar funcionando bien. Pasamos a definir una función que genera primos sacando números al azar.
La haremos un poco más general, pidiéndole que genere varios primos en una llamada.

In [7]:
def _generate_primes(bit_number, number=2):
    # Definimos el mayor y menor número con la cantidad de bits requerida
    upper = 2 ** bit_number - 1
    lower = 2 ** (bit_number - 1)
    
    primes = []
    while len(primes) < number:
        r = random.randint(lower, upper)
        if _is_probably_prime(r):
            primes.append(r)

    return primes

Probemos la función con algo reconocible y luego con algo grande

In [8]:
_generate_primes(1024, 2)

[124375247072980131746990873243134268844337681688399465657646029371412229389818484234809792375929889446032908495012883995815816399025580963109801548982902031315524936237638469696243133763114074112261997854615147015912927320026518860339651972895855282062300887972585242227502124459677985850321847676596967181161,
 168468344407752525603449664523336601572368621706182922249494950212982110370025992019865606276871360500608410590272485628355987865876657825091702120446500242670774218512223947771490399017131334038105939748743758270136834606978550280251251663793058029360593961782988014421514304147712992624646799897234638360611]

Se ve razonable. Vamos ahora a las clases para enviar y recibir.

In [9]:
class RSAReceiver():
    def __init__(self, bit_len):
        P, Q = 0, 0
        while P == Q:
            P, Q = _generate_primes(max(bit_len // 2 + 1, 9))
        self.N = P * Q
        self.block_size = (self.N.bit_length() + 1) // 8
        
        # Encontramos un d al azar co-primo con phi
        phi = (P - 1) * (Q - 1)
        
        r = 2 # r será el MCD entre phi y d
        while r != 1:
            self.d = random.randint(2 ** (bit_len - 1), 2 ** bit_len - 1)
            r, s, t = _extended_euclid(phi, self.d)
            
        e = t % phi
        self.public_key = (
            ((e.bit_length() + 7) // 8).to_bytes(4, 'big') +
            e.to_bytes((e.bit_length() + 7) // 8, 'big') +
            ((self.N.bit_length() + 7) // 8).to_bytes(4, 'big') +
            self.N.to_bytes((self.N.bit_length() + 7) // 8, 'big')
        )
        
        
    def get_public_key(self):
        return self.public_key
    
    def decrypt(self, ciphertext):
        plainbytes = bytearray()
        # Usamos block_size + 1 dado que ese es el tamaño de los bloques encriptados
        for i in range(0, len(ciphertext), self.block_size + 1):
            block = ciphertext[i: i + self.block_size + 1]
            block_number = int.from_bytes(block, 'big')
            plain_block_number = pow(block_number, self.d, self.N)
            plain_block = plain_block_number.to_bytes(self.block_size, 'big')
            plainbytes += plain_block
            
        # El texto no puede tener \x00, lo sacamos para evitar errores en el último bloque.
        return plainbytes.decode('utf-8').replace('\x00', '')

In [10]:
class RSASender():
    def __init__(self, public_key):
        length_e = int.from_bytes(public_key[:4], 'big')
        self.e = int.from_bytes(public_key[4: 4 + length_e], 'big')
        self.N = int.from_bytes(public_key[8 + length_e:], 'big')
        self.block_size = (self.N.bit_length() + 1) // 8
    
    def encrypt(self, message):
        ciphertext = bytearray()
        message_bytes = bytearray(message, 'utf-8')
        
        # Encriptamos cada bloque del mensaje por separado
        for i in range(0, len(message_bytes), self.block_size):
            block = message_bytes[i: i + self.block_size]
            block_number = int.from_bytes(block, 'big')
            encrypted_block_number = pow(block_number, self.e, self.N)
            # Los bloques encriptados usan block_size + 1 bytes
            encrypted_block = encrypted_block_number.to_bytes(self.block_size + 1, 'big')
            ciphertext += encrypted_block
        return ciphertext

Probemos ahora que todo funciona bien con un ejemplo. Creamos un receiver con una llave de largo 1024.

In [11]:
receiver = RSAReceiver(1024)
sender = RSASender(receiver.get_public_key())

Definimos un mensaje para encriptar y decriptar

In [12]:
text = (
    'Being open source means anyone can independently review '
    'the code. If it was closed source, nobody could verify the '
    'security. I think it’s essential for a program of this '
    'nature to be open source.'
)

Lo encriptamos y lo imprimimos en alguna codificación que extienda ASCII (ya que muy probablemente no será un string válido en UTF-8):

In [13]:
cipher = sender.encrypt(text)
print(cipher.decode('cp437'))

Æ≤ⁿ└Ü╬Γ▓6╧╫º±U▀~c╚&Xhèr╔ºïær?°9âK∙Θ╧ç╝Ω·qÜΦ╞╢£╡τ¡╛Ñ4╬2≡]:9w(Nvπê!l<S╟3φ╩]╚÷F+╔Ywæ¿XS{C≈.SÇ┬╤sJú╫ñ½Ñ ─╡m~QYσ⌐■¼1╟á¢τP⌠û+; L/âd╒e·┼Z«ä⌠∙-;cuπñ Iä(n▒¢aóçue╚]0ZûoK7¢αï╫▄dÜ4╬ù:⌡K└]Rú╖Çü·²üazë╣R╠┴rΘ╤─K2╦╫o╓±^¿s₧vjná└ x»ß¡a¬¿Çz°▐5■h°┐╩ └┼█


Vemos que se ve aleatorio. Ahora decriptémoslo.

In [14]:
plaintext = receiver.decrypt(cipher)
print(plaintext)

Being open source means anyone can independently review the code. If it was closed source, nobody could verify the security. I think it’s essential for a program of this nature to be open source.


Yo creo que con este código nos sacamos un 7.

In [15]:
from base64 import b64encode, b64decode
public_key = b64decode('AAAAQQGHaihgiufnjzyLXufDjUCGuaHrsUL+hCF/pMFHPoh+ZVi/2bMFh6oelzElVklsJ9mglyQjJIKAb1JB9mvtaEkLAAAAQQHIuF+wIJw6uzq8uXpW/QmsNjtBJ8HCJJcu2h7sDX18nc2qWYDWTfMiXPmPRvhkkz4A0oXTAMDP9xsxUIjYQNsx')
text = (
    'Being open source means anyone can independently review '
    'the code. If it was closed source, nobody could verify the '
    'security. I think it’s essential for a program of this '
    'nature to be open source.'
)
sender = RSASender(public_key)
cipher = sender.encrypt(text)
print(b64encode(cipher))

b'ALwPm7JXWbqGeIflV8PYgprs6mSgCH2Ydy0rgvFolzY0mczKItlPSHueL54uvDJXIz9pXoHZGAOPWVYYbcwRh3EBl8pi3MraUC2BBFUviMPFwNMwza/QMd5DNG9tH8doHlLRRt+15wLrsIE+m5T8fuM4HHixSNcEoOdN8T++q0PkzQDXL+UgbusiD3J+QPO59aqAB5HFcZ7P5U3fhFS8Qm1vLG8vlIulCby0jGLgjTtLUhFD/QhAof0y4F20gxedQDHwAOIrz6PEoBWnHmwLU0QNN0Rs542RvJ8BeEGhBDS5ZvD0/0Ix3ZqKT6HtP4ugfPD75/5LYGioJBwrg2DXbQucFj8='


## Generando los ejemplos de corrección.
Para la corrección generaremos varios ejemplos para encriptar y varios ejemplos para decriptar. Los guardaremos en un json y usaremos base 64 para que sea legible.

In [16]:
import json
import random
from base64 import b64encode

Los ejemplos para decriptar requieren simplemente un texto y un largo de llave. Se instanciará la clase RSAReceiver del alumno y se le pedirá la llave pública, con lo que se instanciará la clase RSASender presentada más arriba. Luego se encriptará el mensaje con RSASender, se pedirá decriptar el texto y se evaluará si son o no son iguales. Los textos que se utilizarán serán los primeros 15 párrafos del discurso de Denzel Washington del 16 de Mayo de 2011 en la Universidad de Pennsylvania.

In [17]:
possible_key_lengths = [128, 256, 512, 1024]
enc_examples = []
dec_examples = []

In [18]:
content = open('speech.txt').read()
paragraphs = content.split('\n\n')

In [19]:
for par in paragraphs[:15]:
    dec_examples.append({
        'text': par,
        'key_len': random.choice(possible_key_lengths)
    })

Los ejemplos para encriptar requieren tanto una llave pública como un texto. Los textos serán los próximos 15 párrafos del mismo discurso. Las llaves públicas se generan aleatoriamente utilizando la clase RSAReceiver definida arriba.

In [20]:
for par in paragraphs[15:30]:
    receiver = RSAReceiver(random.choice(possible_key_lengths))
    sender = RSASender(receiver.get_public_key())
    enc_examples.append({
        'public_key': b64encode(receiver.get_public_key()).decode(),
        'text': par,
        'ciphertext': b64encode(sender.encrypt(par)).decode()
    })

Escribimos los resultados a un archivo json

In [21]:
grading_file = open('grading.json', 'w')
grading_file.write(
    json.dumps(
        {'encryption': enc_examples, 'decryption': dec_examples},
        indent=4
    )
)
grading_file.close()

## Evaluación
Esta tarea se evaluará de forma simple, contando simplemente la cantidad de ejemplos correctos. En caso de que al decriptar un texto no se eliminen los ceros del último bloque habrá un pequeño descuento.

In [22]:
import json
from base64 import b64decode

In [23]:
examples = json.loads(open('grading.json').read())
enc_points = 0
dec_points = 0
dec_discounts = 0

In [24]:
for enc_example in examples['encryption']:
    public_key = b64decode(enc_example['public_key'])
    ciphertext = b64decode(enc_example['ciphertext'])
    
    # Clase RSASender del alumno
    sender = RSASender(public_key)
    student_ciphertext = sender.encrypt(enc_example['text'])
    
    if student_ciphertext == ciphertext:
        enc_points += 1

Pasamos a los ejemplos para decriptar

In [25]:
for dec_example in examples['decryption']:
    # Clase RSAReceiver del alumno
    receiver = RSAReceiver(dec_example['key_len'])
    
    # Clase RSASender de referencia
    sender = RSASender(receiver.get_public_key())
    
    plaintext = dec_example['text']
    ciphertext = sender.encrypt(dec_example['text'])
    student_plaintext = receiver.decrypt(ciphertext)
    
    if student_plaintext == plaintext:
        dec_points += 1
    elif student_plaintext.replace('\x00', '') == plaintext:
        dec_points += 1
        dec_discounts += 1

Finalmente imprimimos los puntajes y notas

In [26]:
discounted_points = dec_discounts / 10
total_points = enc_points + dec_points - discounted_points
print('--- Resultados evaluación P2 ---')
print(f'Puntos por encriptar: {enc_points} / 15')
print(f'Puntos por decriptar: {enc_points} / 15')
print(f'Puntos descontados por último bloque: {discounted_points}')
print(f'Puntos totales: {total_points} / 30.0')
print(f'Nota: {1 + 6.0 * total_points / 30}')

--- Resultados evaluación P2 ---
Puntos por encriptar: 15 / 15
Puntos por decriptar: 15 / 15
Puntos descontados por último bloque: 0.0
Puntos totales: 30.0 / 30.0
Nota: 7.0


Nos sacamos un 7!